<a href="https://colab.research.google.com/github/tomonari-masada/course2022-sml/blob/main/SVM_vs_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

SEED = 12345

rng = np.random.default_rng(SEED)

# 人工的なデータセットの作成

* データセットXは、3次元ベクトルの集合。インスタンス数は2,000。
* 最初の成分が0または1で、これがそのまま分類の正解ラベルになるように、ターゲットyをデータを作ってある。
* 残りの2つの成分の決め方は、コード通り。

In [2]:
x_zero_list = []
x_zero_time_list = []
x_zero_global_time_list = []

x_one_list = []
x_one_time_list = []
x_one_global_time_list = []

STEP = 1000

for i in range(10):

  x_zero_list.append(np.zeros(100))
  x_zero_time_list.append(rng.integers(-50, 0, size=(100)) + STEP * i)
  x_zero_global_time_list.append(np.full(100, i * STEP))

  x_one_list.append(np.ones(100))
  x_one_time_list.append(rng.integers(0, 50, size=(100)) + STEP * i)
  x_one_global_time_list.append(np.full(100, i * STEP))

x_zero = np.concatenate(x_zero_list)
x_zero_time = np.concatenate(x_zero_time_list)
x_zero_global_time = np.concatenate(x_zero_global_time_list)
x_zero = np.stack([x_zero, x_zero_time, x_zero_global_time]).T

x_one = np.concatenate(x_one_list)
x_one_time = np.concatenate(x_one_time_list)
x_one_global_time = np.concatenate(x_one_global_time_list)
x_one = np.stack([x_one, x_one_time, x_one_global_time]).T

In [3]:
X = np.concatenate([x_zero, x_one])
y = np.array([0] * x_zero.shape[0] + [1] * x_one.shape[0])

* 上のようにして作ったデータセットの、最後の2割の400個を、テストデータとする。

In [4]:
indices = X[:,1] < STEP * 8 - 50
X_train = X[indices]
y_train = y[indices]
X_test= X[~ indices]
y_test = y[~ indices]

* テストデータの3つ目の成分をわざと全て0にセットした、意地悪なテストセットを作る。
* 最初の成分は、相変わらず、正解ラベルそのものになっている。
 * だから、ちゃんと分類してくれていいはず。

In [5]:
X_test_adv = X_test.copy()
X_test_adv[:,2] = 0

* テストデータ以外の部分を、ランダムに、訓練データと検証データに分割。

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, stratify=y_train, random_state=SEED)

# SVMで２値分類

In [7]:
clf_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_svm.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

* 検証データでの分類性能はパーフェクト。

In [8]:
df = pd.DataFrame(X_valid)
df['target'] = y_valid
df['predict'] = clf_svm.predict(X_valid)
df.loc[df.target != df.predict]

,0,1,2,target,predict


* テストデータでの分類性能も、パーフェクト。

In [9]:
df = pd.DataFrame(X_test)
df['target'] = y_test
df['predict'] = clf_svm.predict(X_test)
df.loc[df.target != df.predict]

,0,1,2,target,predict


* 意地悪なテストデータでの分類性能も、パーフェクト。

In [10]:
df = pd.DataFrame(X_test_adv)
df['target'] = y_test
df['predict'] = clf_svm.predict(X_test_adv)
df.loc[df.target != df.predict]

,0,1,2,target,predict


# MLPで２値分類

In [11]:
clf_mlp = MLPClassifier(random_state=SEED, max_iter=2000, n_iter_no_change=2000)
clf_mlp.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(max_iter=2000, n_iter_no_change=2000, random_state=12345)

* 検証データでの分類性能はパーフェクト。

In [12]:
df = pd.DataFrame(X_valid)
df['target'] = y_valid
df['predict'] = clf_mlp.predict(X_valid)
df.loc[df.target != df.predict]

,0,1,2,target,predict


* テストデータでの分類性能も、パーフェクト。

In [13]:
df = pd.DataFrame(X_test)
df['target'] = y_test
df['predict'] = clf_mlp.predict(X_test)
df.loc[df.target != df.predict]

,0,1,2,target,predict


* 意地悪なテストデータでの分類性能は、全然ダメ。
 * 全てラベル1と予測してしまっている。

In [14]:
df = pd.DataFrame(X_test_adv)
df['target'] = y_test
df['predict'] = clf_mlp.predict(X_test_adv)
df.loc[df.target != df.predict]

,0,1,2,target,predict
0,0.0,7995.0,0.0,0,1
1,0.0,7997.0,0.0,0,1
2,0.0,7962.0,0.0,0,1
3,0.0,7994.0,0.0,0,1
4,0.0,7954.0,0.0,0,1
...,...,...,...,...,...
195,0.0,8960.0,0.0,0,1
196,0.0,8991.0,0.0,0,1
197,0.0,8960.0,0.0,0,1
198,0.0,8995.0,0.0,0,1


In [15]:
(df.predict == 1).sum()

400